# 🎯 ULEPSZAMY NASZE MLP! — czyli jak nie zbudować sieci, która niczego się nie nauczy

## 🔍 O co chodzi?

W tej części kursu zajmiemy się **najczęstszymi problemami**, jakie pojawiają się przy trenowaniu prostych sieci neuronowych (MLP — Multi-Layer Perceptron). Nawet jeśli Twoja sieć działa, to...

> ...czy robi to **dobrze**?
> Czy **uczy się szybko**?
> Czy nie **przeucza się**?
> Czy naprawdę rozumiesz **co się dzieje w środku**?

---

## 🚨 Problemy, z którymi się zmierzymy:

- sieć **uczy się zbyt wolno**
- **loss skacze** albo w ogóle nie spada
- model **zapamiętuje dane treningowe**, ale nie radzi sobie na nowych
- wyniki **losowe jak totolotek**

---

## 🧠 Co dziś dodamy do naszego kodu?

Każdy trick, który dziś zobaczysz, to **kilka linijek kodu**, ale może:
- **zwiększyć dokładność**
- **przyspieszyć naukę**
- **zmniejszyć overfitting**
- **poprawić stabilność** całego procesu

Oto nasze „supermoce”:

| 🛠️ Technika               | 🎯 Co robi?                            |
|--------------------------|----------------------------------------|
| `Dropout`                | Gasi część neuronów, by model nie "oszukiwał" |
| `Weight Decay`           | Kara za zbyt duże wagi (regularizacja) |
| `Learning Rate Decay`    | Zmniejsza krok ucznia w czasie         |
| `Gradient Clipping`      | Powstrzymuje gradienty przed "eksplozją" |
| `Early Stopping`         | Kończymy trenowanie w dobrym momencie |
| `Data Augmentation`      | Sprawia, że dane są bardziej zróżnicowane |

---

## 💡 Cel

> Chcemy zbudować **prostą sieć**, ale uczącą się **mądrze**.  
> Zrozumieć, **co się psuje** — i jak to naprawić.
> Wykorzystamy do tego sieć, którą zrobiliśmy na poprzednich ćwiczeniach

---



In [ ]:
# 📦 Import bibliotek
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from tqdm import tqdm

# 🔧 Ustawienie urządzenia
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Używane urządzenie: {device}")

# 📊 Ocena modelu na danych testowych
def test_model(model, testloader):

  correct = 0
  total = 0

  with torch.no_grad():
      for inputs, labels in testloader:
          inputs, labels = inputs.to(device), labels.to(device)
          outputs = model(inputs)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print(f"Dokładność na zbiorze testowym: {100 * correct / total:.2f}%")
  return 100 * correct / total


# 📥 Wczytanie i przetwarzanie danych MNIST
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

BATCH_SIZE = 64

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)


# 🧠 Definicja sieci neuronowej przy użyciu nn.Sequential
model = nn.Sequential(
    nn.Flatten(),                # Spłaszczenie obrazu 28x28 -> 784
    nn.Linear(784, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 5

for epoch in tqdm(range(epochs)):
    for batch_id, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = loss_fn(outputs, labels)


        if batch_id % 1000 == 0:
            print(f"[Batch {batch_id}] Loss: {loss.mean():.4f}")

        loss.backward()
        optimizer.step()

test_model(model, testloader)

Używane urządzenie: cuda


  0%|          | 0/5 [00:00<?, ?it/s]

[Batch 0] Loss: 2.3213


 20%|██        | 1/5 [00:12<00:48, 12.10s/it]

[Batch 0] Loss: 0.1031


...

 80%|████████  | 4/5 [00:48<00:12, 12.04s/it]

[Batch 0] Loss: 0.2679


100%|██████████| 5/5 [01:00<00:00, 12.03s/it]


Dokładność na zbiorze testowym: 96.13%


96.13

## 1. Dropout — czyli nie polegaj tylko na jednym neuronie

Czasem sieć za bardzo przywiązuje się do konkretnych wag czy neuronów. Dropout losowo "wyłącza" część z nich podczas treningu. Dzięki temu sieć uczy się bardziej ogólnych cech i mniej się przeucza.


In [ ]:
model_with_dropout = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 128),
    nn.ReLU(),
    nn.Dropout(0.3),  # 👈 dodajemy dropout 30%
    nn.Linear(128, 10)
).to(device)


# dodajemy w optimizer weight_decay
optimizer = optim.Adam(model_with_dropout.parameters(), lr=0.001)

for epoch in tqdm(range(epochs)):
    for batch_id, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model_with_dropout(inputs)
        loss = loss_fn(outputs, labels)


        if batch_id % 1000 == 0:
            print(f"[Batch {batch_id}] Loss: {loss.mean():.4f}")

        loss.backward()
        optimizer.step()


test_model(model_with_dropout, testloader)

  0%|          | 0/5 [00:00<?, ?it/s]

[Batch 0] Loss: 2.3934


 20%|██        | 1/5 [00:12<00:48, 12.20s/it]

[Batch 0] Loss: 0.2629


 40%|████      | 2/5 [00:24<00:36, 12.06s/it]

...

 80%|████████  | 4/5 [00:48<00:12, 12.03s/it]

[Batch 0] Loss: 0.2458


100%|██████████| 5/5 [01:00<00:00, 12.03s/it]


Dokładność na zbiorze testowym: 94.19%


94.19

## 2. Weight Decay (L2) — czyli mniej szalonych wag

Zbyt duże wagi = ryzyko, że model zapamięta dane zamiast się ich nauczyć. Weight Decay delikatnie karze duże wagi i pomaga lepiej generalizować.


In [ ]:

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4) # 👈 Nowa rzecz w optimizerze

for epoch in tqdm(range(epochs)):
    for batch_id, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = loss_fn(outputs, labels)


        if batch_id % 1000 == 0:
            print(f"[Batch {batch_id}] Loss: {loss.mean():.4f}")

        loss.backward()
        optimizer.step()


test_model(model, testloader)


  0%|          | 0/5 [00:00<?, ?it/s]

[Batch 0] Loss: 0.0654


 20%|██        | 1/5 [00:12<00:48, 12.06s/it]

[Batch 0] Loss: 0.0652


 40%|████      | 2/5 [00:24<00:36, 12.01s/it]

...

 80%|████████  | 4/5 [00:48<00:12, 12.09s/it]

[Batch 0] Loss: 0.0202


100%|██████████| 5/5 [01:00<00:00, 12.09s/it]


Dokładność na zbiorze testowym: 96.77%


96.77

## 3. Learning Rate Decay — czyli zwalniamy przy finiszu

Na początku warto robić duże kroki (duży learning rate), ale z czasem warto zwolnić, żeby nie przegapić celu. Learning Rate Decay to strategia zmniejszania kroku w miarę postępu treningu.


In [ ]:


scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5) # 👈 NOWOŚĆ
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in tqdm(range(epochs)):
    for batch_id, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = loss_fn(outputs, labels)


        if batch_id % 1000 == 0:
            print(f"[Batch {batch_id}] Loss: {loss.mean():.4f}")

        loss.backward()
        optimizer.step()
    scheduler.step() # Po zakończonym Batchu aktualizujemy nasz learning_rate


test_model(model, testloader)

  0%|          | 0/5 [00:00<?, ?it/s]

[Batch 0] Loss: 0.0625


 20%|██        | 1/5 [00:12<00:49, 12.27s/it]

[Batch 0] Loss: 0.0440


 40%|████      | 2/5 [00:24<00:36, 12.22s/it]

...

 80%|████████  | 4/5 [00:48<00:12, 12.22s/it]

[Batch 0] Loss: 0.0495


100%|██████████| 5/5 [01:01<00:00, 12.22s/it]


Dokładność na zbiorze testowym: 97.47%


97.47

## 4. Gradient Clipping — czyli bez wybuchających gradientów
Czasem gradienty mogą osiągać absurdalnie wysokie wartości — szczególnie na początku. To może sprawić, że sieć zamiast się uczyć, będzie tylko „krzyczeć w panice”.

Gradient Clipping to jak kaganiec — ogranicza długość gradientów i pozwala trenować stabilniej.

In [ ]:
for epoch in tqdm(range(epochs)):
    for batch_id, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = loss_fn(outputs, labels)


        if batch_id % 1000 == 0:
            print(f"[Batch {batch_id}] Loss: {loss.mean():.4f}")

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # 👈 To tutaj ustalamy maksymalny gradient
        optimizer.step()


test_model(model, testloader)


  0%|          | 0/5 [00:00<?, ?it/s]

[Batch 0] Loss: 0.1802


 20%|██        | 1/5 [00:12<00:49, 12.47s/it]

[Batch 0] Loss: 0.0446


 40%|████      | 2/5 [00:24<00:37, 12.43s/it]

...

 80%|████████  | 4/5 [00:49<00:12, 12.31s/it]

[Batch 0] Loss: 0.0032


100%|██████████| 5/5 [01:01<00:00, 12.31s/it]


Dokładność na zbiorze testowym: 97.54%


97.54

## 5. Early Stopping — czyli wiesz kiedy przestać
Nie zawsze więcej znaczy lepiej.
Czasem sieć zaczyna się „psuć” po kilku epokach, bo po prostu za długo się uczy.
Wtedy dobrze mieć Early Stopping — jeśli dokładność nie poprawia się przez kilka epok, po prostu przerywamy trening.

In [ ]:
best_acc = 1e8 # 👈 zmienna do trzymania najlepszego lossu
epochs_no_improve = 0 # 👈 zmienna mówiąca ile epoch minęło bez poprawy
early_stop_patience = 3  # 👈 zmienna mówiąca ile epoch tolerujemy (takich podczas których może nie być poprawy)

for epoch in tqdm(range(epochs)):
    for batch_id, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = loss_fn(outputs, labels)


        if batch_id % 1000 == 0:
            print(f"[Batch {batch_id}] Loss: {loss.mean():.4f}")

        loss.backward()
        optimizer.step()

    if loss.item() < best_acc:  # 👈 Cała logika odnośnie Early Stoppingu
        best_acc = loss.item()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= early_stop_patience:
        print(f"Early stopping triggered after {epoch + 1} epochs.")
        break


test_model(model, testloader)

  0%|          | 0/5 [00:00<?, ?it/s]

[Batch 0] Loss: 0.0012


 20%|██        | 1/5 [00:11<00:47, 11.95s/it]

[Batch 0] Loss: 0.0152


 40%|████      | 2/5 [00:23<00:35, 11.88s/it]

...

 80%|████████  | 4/5 [00:48<00:12, 12.15s/it]

[Batch 0] Loss: 0.0181


100%|██████████| 5/5 [01:00<00:00, 12.12s/it]


Dokładność na zbiorze testowym: 97.34%


97.34

## 6. Data Augmentation — czyli ucz się na więcej sposobów

Jeśli pokazujesz modelowi tylko 1 wersję danych, łatwo je zapamięta. Augmentacja (np. obracanie, szum, przesunięcia) tworzy sztucznie nowe przykłady — sieć uczy się uogólniać.


In [ ]:
transform_aug = transforms.Compose([
    transforms.RandomRotation(degrees=10),  # 👈 Augmentacja
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 📥 Wczytanie i przetwarzanie danych MNIST
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

BATCH_SIZE = 64

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_aug) # 👈 Tutaj zmieniamy nasz transform w pobieraniu Datasetu
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)


optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in tqdm(range(epochs)):
    for batch_id, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = loss_fn(outputs, labels)


        if batch_id % 1000 == 0:
            print(f"[Batch {batch_id}] Loss: {loss.mean():.4f}")

        loss.backward()
        optimizer.step()


test_model(model, testloader)

  0%|          | 0/5 [00:00<?, ?it/s]

[Batch 0] Loss: 0.0500


 20%|██        | 1/5 [00:17<01:09, 17.40s/it]

[Batch 0] Loss: 0.0918


 40%|████      | 2/5 [00:34<00:50, 16.97s/it]

...

 80%|████████  | 4/5 [01:07<00:16, 16.90s/it]

[Batch 0] Loss: 0.0055


100%|██████████| 5/5 [01:24<00:00, 16.90s/it]


Dokładność na zbiorze testowym: 97.50%


97.5

## Najlepsze na koniec!

Czyli czy warto zawsze wszystkie z tych metod stosować ze sobą

In [ ]:
transform_aug = transforms.Compose([
    transforms.RandomRotation(degrees=10),  # 👈 augmentacja
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

BATCH_SIZE = 64

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_aug)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)


model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 10)
).to(device)

best_acc = 0
epochs_no_improve = 0
early_stop_patience = 3  # epoki bez poprawy

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)

for epoch in tqdm(range(epochs)):
    for batch_id, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = loss_fn(outputs, labels)


        if batch_id % 1000 == 0:
            print(f"[Batch {batch_id}] Loss: {loss.mean():.4f}")

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
    scheduler.step()

    if loss.mean() < best_acc:
        best_acc = loss.mean()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= early_stop_patience:
        print(f"Early stopping triggered after {epoch + 1} epochs.")
        break


test_model(model, testloader)

  0%|          | 0/5 [00:00<?, ?it/s]

[Batch 0] Loss: 2.3166


 20%|██        | 1/5 [00:17<01:08, 17.10s/it]

[Batch 0] Loss: 0.2557


 40%|████      | 2/5 [00:34<00:51, 17.10s/it]

[Batch 0] Loss: 0.2999


 40%|████      | 2/5 [00:52<01:18, 26.01s/it]

Early stopping triggered after 3 epochs.


Dokładność na zbiorze testowym: 94.73%


94.73